In [ ]:
# #To extract all the links on a website, write the following code in the console:

var x = document.querySelectorAll("a");
var myarray = []
for (var i=0; i<x.length; i++){
var nametext = x[i].textContent;
var cleantext = nametext.replace(/\s+/g, ' ').trim();
var cleanlink = x[i].href;
myarray.push([cleantext,cleanlink]);
};
function make_table() {
    var table = '<table><thead><th>Name</th><th>Links</th></thead><tbody>';
   for (var i=0; i<myarray.length; i++) {
            table += '<tr><td>'+ myarray[i][0] + '</td><td>'+myarray[i][1]+'</td></tr>';
    };
 
    var w = window.open("");
w.document.write(table); 
}
make_table()

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv
import os
import pandas as pd

In [2]:
# Read the CSV file into a dataframe
df = pd.read_csv('Links.csv')

# Extract the 'URL' column into the target_urls variable
target_urls = df['URL'].tolist()

In [3]:
# Set Chrome options to accept cookies
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--profile-directory=Default")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--enable-automation")
chrome_options.add_argument("--enable-javascript")
chrome_options.add_argument("--accept-langauge=en")
chrome_options.add_argument("--headless")  # Run Chrome in headless mode, i.e., without opening a browser window

# Set the path to the ChromeDriver executable
chrome_driver_path = '/path/to/chromedriver'

# Initialize the Chrome driver with the options
driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)

C:\Users\nites\AppData\Local\Temp\ipykernel_13784\3651952518.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)


In [4]:
# Set the screen resolution (you can adjust this as per your requirement)
screen_width = 1920
screen_height = 1080
driver.set_window_size(screen_width, screen_height)

# Create a directory to store the website images
output_dir = 'images'
os.makedirs(output_dir, exist_ok=True)

In [5]:
# Create a CSV file to store the classification results
csv_file = 'classification_results.csv'
def classify_content(content):
    # Convert the content to lowercase for case-insensitive matching
    content = content.lower()

    # Define the keywords for different categories
    e_commerce_keywords = ['Buy', 'Shop', 'Product', 'Purchase', 'Order', 'Add to Cart', 'Checkout', 'Discount', 'Sale', 'Deal', 'Shipping', 'Delivery', 'Online Store', 'Shopping', 'Ecommerce', 'Shop Now', 'Best Sellers', 'New Arrivals', 'Customer Reviews', 'Shopping Cart']
    news_keywords = ['News', 'Headlines', 'Breaking news', 'Current affairs', 'World news', 'Local news', 'International news', 'Politics', 'Business', 'Technology', 'Science', 'Health', 'Entertainment', 'Sports', 'Opinion', 'Editorials', 'Features', 'Investigative reporting', 'Interviews', 'Analysis']
    blog_keywords = ['Blog', 'Article', 'Post', 'Opinion', 'Thoughts', 'Personal', 'Experience', 'Journal', 'Tips', 'How-to', 'DIY', 'Lifestyle', 'Fashion', 'Travel', 'Food', 'Fitness', 'Health', 'Parenting', 'Beauty', 'Photography']
    entertainment_keywords=['Entertainment', 'Movies', 'TV Shows', 'Music', 'Celebrities', 'Gossip', 'Film Reviews', 'Celebrity News', 'Concerts', 'Events', 'Humor', 'Comedy', 'Games', 'Online Streaming', 'Pop Culture', 'Entertainment News']
    educational_keywords = ['Education', 'Online Learning', 'Courses', 'Tutorials', 'Study Materials', 'Research', 'Academic', 'E-Learning', 'Knowledge', 'Educational Resources', 'School', 'University', 'Student', 'Teacher', 'Learning Platform']
    govt_websites = ['Government', 'act','Official', 'Public Services', 'Government Programs', 'Policies', 'Laws', 'Regulations', 'Public Records', 'Citizenship', 'Taxes', 'Healthcare', 'Education', 'Employment', 'Visa', 'Passport', 'Government Agencies']
    
    try:
        # Check for keywords in the content to determine the category
        if any(keyword in content for keyword in e_commerce_keywords):
            return 'E-commerce'
        elif any(keyword in content for keyword in news_keywords):
            return 'News'
        elif any(keyword in content for keyword in blog_keywords):
            return 'Blog'
        elif any(keyword in content for keyword in entertainment_keywords):
            return 'Entertainment'
        elif any(keyword in content for keyword in educational_keywords):
            return 'Educational'
        elif any(keyword in content for keyword in govt_keywords):
            return 'Government'
        else:
            return 'Unknown'

    except Exception as e:
        return 'Unknown'

In [7]:
# Loop through the target URLs and scrape content, capture screenshots, and perform classification
results = []
for i, url in enumerate(target_urls):
    try:
        # Open the URL
        driver.get(url)

        time.sleep(5)
        
        # Handle cookie consent pop-ups (if any)
        try:
            cookie_button = driver.find_element(By.XPATH, '//*[contains(text(), "Accept Cookies")]')
            cookie_button.click()
            time.sleep(2)
        except:
            pass

        # Extract content from the webpage
        content_elements = driver.find_elements(By.XPATH, '//p | //h1 | //h2 | //h3')  # Search for <p>, <h1>, <h2>, <h3> tags
        content = None

        # Iterate through the content elements
        for element in content_elements:
            element_text = element.text.strip()
            if element_text:
                content = ' '.join([element_text for element_text in element_text.split() if element_text.strip()])
                break
        
        # Perform content classification
        category = classify_content(content)
        # Capture the screenshot of the entire page
        screenshot_file = f'{output_dir}/screenshot_{i}.png'
        driver.save_screenshot(screenshot_file)
        print(f'Screenshot captured for {url} and saved as {screenshot_file}')
#         if category != 'Unknown':
#             # Capture the screenshot of the entire page
#             screenshot_file = f'{output_dir}/screenshot_{i}.png'
#             driver.save_screenshot(screenshot_file)
#             print(f'Screenshot captured for {url} and saved as {screenshot_file}')
#         else:
#             print(f'Screenshot could not be captured for {url}')
#             screenshot_file = None

        results.append((url, category, content, screenshot_file))

    except Exception as e:
        print(f'Error processing {url}: {str(e)}')

# Quit the driver and close the browser
driver.quit()

Screenshot captured for https://www.bestbuy.com and saved as images/screenshot_0.png
Screenshot captured for http://en.people.cn/ and saved as images/screenshot_1.png


In [8]:
# Write the results to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Website', 'Category', 'Content', 'Screenshot File'])
    for result in results:
        # Encode the content to handle non-ASCII characters
        encoded_content = result[2].encode('ascii', 'ignore').decode('ascii')
        # Create a new tuple with the encoded content
        encoded_result = (result[0], result[1], encoded_content, result[3])
        writer.writerow(encoded_result)

print(f'Classification results saved to {csv_file}')

Classification results saved to classification_results.csv


In [ ]:
#Pre Processing the screenshots taken
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import os

# Directory path of the website screenshots
screenshot_dir = 'website_images'

# Output directory path for preprocessed screenshots
preprocessed_dir = 'preprocessed_images'
os.makedirs(preprocessed_dir, exist_ok=True)

# Define the desired size and resolution for the preprocessed screenshots
desired_width = 1280
desired_height = 720

# Function to preprocess a single screenshot
def preprocess_screenshot(filename):
    if filename.endswith('.png'):
        # Open the screenshot image
        image_path = os.path.join(screenshot_dir, filename)
        image = Image.open(image_path)
        
        # Resize the image while maintaining the aspect ratio
        image.thumbnail((desired_width, desired_height))

        # Determine the cropping area to remove irrelevant sections (e.g., browser tabs, sidebars)
        # Adjust the cropping dimensions according to your specific requirements
        crop_left = 0
        crop_top = 100
        crop_right = image.width
        crop_bottom = image.height - 100

        # Crop the image to focus on the main content area
        image = image.crop((crop_left, crop_top, crop_right, crop_bottom))
        
        # Convert the image to RGB color mode if it's not already
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Save the preprocessed image to the output directory
        output_path = os.path.join(preprocessed_dir, filename)
        image.save(output_path)
        print(f'Preprocessed image saved: {output_path}')

# Get the list of screenshot files in the directory
screenshot_files = [filename for filename in os.listdir(screenshot_dir) if filename.endswith('.png')]

# Process the screenshots using parallel processing
with ThreadPoolExecutor() as executor:
    executor.map(preprocess_screenshot, screenshot_files)